In [11]:
import graphdb_client
import logging 
import json
import numpy as np


logging.basicConfig(level=logging.DEBUG, format='[%(levelname)s] %(lineno)s %(message)s',)
g = graphdb_client.gc(host = 'http://localhost:8010')

### Calculate betweenness

In [12]:
rc = g.set_current_graph('hw3_demo_1')
print (rc)

current graph set to hw3_demo_1


In [13]:
rc1 = g.get_neighbor_out(vertex_id = '238', vertex_label='MOVIE', neighbor_label = ['ACTOR','DIRECTOR'])
layer_one = json.loads(rc1)['data']['vertices']
total_id = ['238']
total_label = ['MOVIE']

#print rc1
for item_one in layer_one:
    id1 = item_one['id']
    label1 = item_one['label']
    total_id.append(id1)
    total_label.append(label1) 
    rc2 = g.get_neighbor_out(vertex_id = id1, vertex_label=label1, neighbor_label = ['MOVIE','ACTOR','DIRECTOR']) 
    layer_two = json.loads(rc2)['data']['vertices']
    for item_two in layer_two:
        id2 = item_two['id']
        label2 = item_two['label']
        if id2 not in total_id:
            total_id.append(id2)
            total_label.append(label2) 

[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "GET /graphs/hw3_demo_1/vertices/238/MOVIE/outV?vertex_label=ACTOR;DIRECTOR HTTP/1.1" 200 1369
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "GET /graphs/hw3_demo_1/vertices/3085/ACTOR/outV?vertex_label=MOVIE;ACTOR;DIRECTOR HTTP/1.1" 200 3317
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "GET /graphs/hw3_demo_1/vertices/238/MOVIE/outV?vertex_label=MOVIE;ACTOR;DIRECTOR HTTP/1.1" 200 1369
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "GET /graphs/hw3_demo_1/vertices/3084/ACTOR/outV?vertex_label=MOVIE;ACTOR;DIRECTOR HTTP/1.1" 200 2743
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "GET /graphs/hw3_demo_1/vertices/3092/ACTOR/outV?vertex_label=MOVIE;ACTOR;DIRECTOR HTTP/1.1" 200 4188
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "GET /graphs/hw3_demo_1/vertices/1776/DIRECTOR/outV?vertex_label=MOVIE;ACTOR;DIRECTOR HTTP/1.1" 

In [14]:
  

dirt = 0
rc = g.get_subgraph(vertex_ids = total_id, vertex_labels = total_label, edge_labels = ['ACT','DIR'])
vertices = json.loads(rc)['data']['vertices']
dir=[0]*len(total_id)
for start in vertices:
    start_id = start['id']
    start_label = start['label']
    for end in vertices:
        end_id = end['id']
        end_label = end['label']
        if start_id != end_id:
            rc = g.get_path(source_id=start_id, source_label=start_label, target_id = end_id, target_label = end_label, edge_labels = ['ACT', 'DIR'], depth = 3)
            ver = json.loads(rc)
            dirt = dirt + 1
            if ver['status'] != 'failure':
                ver = ver['data']['vertices']
                for ve in ver:
                    if ve['id'] in total_id and total_label[total_id.index(ve['id'])] == 'DIRECTOR':
                        dir[total_id.index(ve['id'])] += 1
 
    betweenness =[a/(dirt + 0.0) for a in dir]
    most_imp = betweenness.index(max(betweenness))
print betweenness

rc_director = g.get_vertex(vertex_label = total_label[most_imp], vertex_id = total_id[most_imp])

print json.loads(rc_director)['data']['vertices'][0]['properties']

[DEBUG] 684 {'param': '{"edge_labels": ["ACT", "DIR"], "vertices": [{"id": "238", "label": "MOVIE"}, {"id": "3085", "label": "ACTOR"}, {"id": "9268", "label": "MOVIE"}, {"id": "5902", "label": "MOVIE"}, {"id": "87428", "label": "MOVIE"}, {"id": "8592", "label": "MOVIE"}, {"id": "11665", "label": "MOVIE"}, {"id": "13685", "label": "MOVIE"}, {"id": "22794", "label": "MOVIE"}, {"id": "109451", "label": "MOVIE"}, {"id": "10719", "label": "MOVIE"}, {"id": "1619", "label": "MOVIE"}, {"id": "238", "label": "MOVIE"}, {"id": "3084", "label": "ACTOR"}, {"id": "3092", "label": "ACTOR"}, {"id": "1776", "label": "DIRECTOR"}, {"id": "3086", "label": "ACTOR"}, {"id": "3090", "label": "ACTOR"}, {"id": "3142", "label": "ACTOR"}, {"id": "3091", "label": "ACTOR"}, {"id": "3087", "label": "ACTOR"}, {"id": "1158", "label": "ACTOR"}, {"id": "3088", "label": "ACTOR"}, {"id": "3084", "label": "ACTOR"}, {"id": "654", "label": "MOVIE"}, {"id": "9306", "label": "MOVIE"}, {"id": "1452", "label": "MOVIE"}, {"id": 

[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "3142", "label": "ACTOR"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "238", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 596
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "3091", "label": "ACTOR"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "238", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 596
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "3087", "label": "ACTOR"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "238", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 598
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "1158", "label": "ACTOR"}, "edge_labels": ["ACT", "DIR"

[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1033
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "1776", "label": "DIRECTOR"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "238", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 595
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "2148", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "238", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 982
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "6114", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "238", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 985
[DEBUG] 700 {'param': '{"depth": 3, "vert

[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 989
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "20766", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "238", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1012
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "2055", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "238", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 984
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "12193", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "238", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/

[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 618
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "22794", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "3085", "label": "ACTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 607
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "109451", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "3085", "label": "ACTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 622
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "10719", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "3085", "label": "ACTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path H

...

[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1221
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "12150", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "3088", "label": "ACTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1209
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "9504", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "3088", "label": "ACTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1199
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "298", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "3088", "label": "ACTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HT

[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1241
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "3142", "label": "ACTOR"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "935", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1239
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "3091", "label": "ACTOR"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "935", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1239
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "3087", "label": "ACTOR"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "935", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/

[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 50
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "1776", "label": "DIRECTOR"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "935", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1238
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "2148", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "935", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 50
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "6114", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "935", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1

[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 50
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "20766", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "935", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 50
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "2055", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "935", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 50
[DEBUG] 700 {'param': '{"depth": 3, "vertex_target": {"id": "12193", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "935", "label": "MOVIE"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1"

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06903906600808263, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
8908
15
1776
[{u'name': u'FrancisFordCoppola'}, {u'gender': 2}]


In [15]:
# For closeness
dir=[0]*len(total_id) 
rc = g.get_subgraph(vertex_ids = total_id, vertex_labels = total_label, edge_labels = ['ACT','DIR'])
for start in json.loads(rc)['data']['vertices']:
    if start['label'] == 'DIRECTOR':
        start_id = start['id']
        start_label = start['label']
        for end in json.loads(rc)['data']['vertices']:
            end_id = end['id']
            end_label = end['label']
            if start_id != end_id:
                rc = g.get_path(source_id=start_id, source_label=start_label, target_id = end_id, target_label = end_label, edge_labels = ['ACT', 'DIR'], depth = 4);
                ver = json.loads(rc)['statistics']['num_edges']
                dir[total_id.index(start_id)] += ver
                


closeness = [1.0/a if a != 0 else 0 for a in dir]
most_imp = np.argmax(closeness)
print dir
print closeness
print total_label[most_imp]
print total_id[most_imp]
rc_director = g.get_vertex(vertex_label = total_label[most_imp], vertex_id = total_id[most_imp])

print json.loads(rc_director)['data']['vertices'][0]['properties']

[DEBUG] 684 {'param': '{"edge_labels": ["ACT", "DIR"], "vertices": [{"id": "238", "label": "MOVIE"}, {"id": "3085", "label": "ACTOR"}, {"id": "9268", "label": "MOVIE"}, {"id": "5902", "label": "MOVIE"}, {"id": "87428", "label": "MOVIE"}, {"id": "8592", "label": "MOVIE"}, {"id": "11665", "label": "MOVIE"}, {"id": "13685", "label": "MOVIE"}, {"id": "22794", "label": "MOVIE"}, {"id": "109451", "label": "MOVIE"}, {"id": "10719", "label": "MOVIE"}, {"id": "1619", "label": "MOVIE"}, {"id": "238", "label": "MOVIE"}, {"id": "3084", "label": "ACTOR"}, {"id": "3092", "label": "ACTOR"}, {"id": "1776", "label": "DIRECTOR"}, {"id": "3086", "label": "ACTOR"}, {"id": "3090", "label": "ACTOR"}, {"id": "3142", "label": "ACTOR"}, {"id": "3091", "label": "ACTOR"}, {"id": "3087", "label": "ACTOR"}, {"id": "1158", "label": "ACTOR"}, {"id": "3088", "label": "ACTOR"}, {"id": "3084", "label": "ACTOR"}, {"id": "654", "label": "MOVIE"}, {"id": "9306", "label": "MOVIE"}, {"id": "1452", "label": "MOVIE"}, {"id": 

[DEBUG] 700 {'param': '{"depth": 4, "vertex_target": {"id": "3090", "label": "ACTOR"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "1776", "label": "DIRECTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 804
[DEBUG] 700 {'param': '{"depth": 4, "vertex_target": {"id": "3142", "label": "ACTOR"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "1776", "label": "DIRECTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 802
[DEBUG] 700 {'param': '{"depth": 4, "vertex_target": {"id": "3091", "label": "ACTOR"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "1776", "label": "DIRECTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 802
[DEBUG] 700 {'param': '{"depth": 4, "vertex_target": {"id": "3087", "label": "ACTOR"}, "edge_labels": [

[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1218
[DEBUG] 700 {'param': '{"depth": 4, "vertex_target": {"id": "253235", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "1776", "label": "DIRECTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1254
[DEBUG] 700 {'param': '{"depth": 4, "vertex_target": {"id": "2148", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "1776", "label": "DIRECTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 594
[DEBUG] 700 {'param': '{"depth": 4, "vertex_target": {"id": "6114", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "1776", "label": "DIRECTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analyti

[DEBUG] 700 {'param': '{"depth": 4, "vertex_target": {"id": "2001", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "1776", "label": "DIRECTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1210
[DEBUG] 700 {'param': '{"depth": 4, "vertex_target": {"id": "20766", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "1776", "label": "DIRECTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1233
[DEBUG] 700 {'param': '{"depth": 4, "vertex_target": {"id": "2055", "label": "MOVIE"}, "edge_labels": ["ACT", "DIR"], "vertex_source": {"id": "1776", "label": "DIRECTOR"}}'}
[INFO] 207 Starting new HTTP connection (1): localhost
[DEBUG] 387 "POST /graphs/hw3_demo_1/analytics/path HTTP/1.1" 200 1205
[DEBUG] 700 {'param': '{"depth": 4, "vertex_target": {"id": "12193", "label": "MOVIE"}, "edge_label

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 244, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.004098360655737705, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
DIRECTOR
1776
[{u'name': u'FrancisFordCoppola'}, {u'gender': 2}]
